### Сбор фотографий с привязкой к гео-координатам

Загрузка фотографий с привязкой к координатам с помощью API VK для доступа к общедоступным фотографиям photos.search.

In [75]:
import numpy as np
from datetime import datetime, timedelta, date, time
import time
import random
import requests

Определяем интересующие границы, глубину поиска по времени и число шагов в каждом измерении.
Объявим функцию для вычисления расстояния по координатам.

In [76]:
top_left = (58.689155, 36.575942)
top_right = (58.689155, 40.421157)
bottom_left = (53.81445180, 36.575942)
bottom_right = (53.81445180, 40.421157)

years = 3
timeperiod = (int((datetime.now() - timedelta(days=365 * years)).timestamp()), int(datetime.now().timestamp()))

steps = 100

In [77]:
import math

def distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в км
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    a = math.sin(dLat/2) * math.sin(dLat/2) + \
        math.sin(dLon/2) * math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

lat1, lon1 = top_right
lat2, lon2 = bottom_right
print(distance(lat1, lon1, lat2, lon2))


542.0422647379957


In [78]:
lat_values = np.linspace(bottom_left[0], top_left[0], steps)
lon_values = np.linspace(top_left[1], top_right[1], steps)
radius = math.floor(distance(lat_values[0], lon_values[0], lat_values[1], lon_values[1]) * 1000 / 2)

In [79]:
def format_date(timestamp):
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')

In [80]:
print(f"""
      Getting photos
          from lat: {lat_values[0]}, lon: {lon_values[0]}
          to lat: {lat_values[-1]}, lon: {lon_values[-1]} 
          with {steps} steps each 
          with radius {radius} meters
          between {format_date(timeperiod[0])} and {format_date(timeperiod[1])}
""")


      Getting photos
          from lat: 53.8144518, lon: 36.575942
          to lat: 58.689155, lon: 40.421157 
          with 100 steps each 
          with radius 3019 meters
          between 2021-01-19 and 2024-01-19



Задаем токен для доступа к API и объявляем функцию для запроса - здесь обычный get-интерфейс с url параметрами.

In [16]:
VK_ACCESS_TOKEN = ''
VK_VERSION = ''

In [17]:
def get_user_photos(geo, radius, timeperiod, offset=0):
    params = {
        'lat': geo[0],
        'long': geo[1],
        'count': '1000',
        'offset': offset,
        'radius': radius,
        'start_time': timeperiod[0],
        'end_time': timeperiod[1],
        'access_token': VK_ACCESS_TOKEN,
        'v': VK_VERSION,
        'sort': 0
    }
    return requests.get("https://api.vk.com/method/photos.search",
                        params=params, verify=True).json()

Последовательно проходимс по всем координатам, сохраняя в переменную items данные об изображениях. 

In [ ]:
items = []

for lat in lat_values:
    for long in lon_values:
        resp = get_user_photos((lat, long), radius, timeperiod)
        new_items = resp['response']['items']
        period = round(random.random() * 5);
        print(f'For {lat}, {long} got {len(new_items)} items, sleep for {period}s')
        time.sleep(period)
        items = items + new_items

Сохраняем изображения, имеющие привязку к координатам.

In [ ]:
print_count = 100

for i, img in enumerate(items):
    if i % print_count == 0:
        period = round(random.random() * 10);
        print(f'Processed {i} images, sleep for {period}s')
        time.sleep(period)
    if 'lat' in img and 'long' in img:
        lat = img['lat']
        long = img['long']
        url = img['sizes'][-1]['url']
        response = requests.get(url)
        with open(f'raw_images/{lat},{long}.jpg', 'wb') as f:
            f.write(response.content)

Итого имеем на выходе папку с изображениями, названия которых соответствуют значениям широты и долготы, где был сделан снимок.